# Scraping trials for Publishers!

In [68]:
import copy
import requests
import scrapelib
from lxml import etree
from bs4 import BeautifulSoup

from selenium import webdriver

## helpers

In [25]:
def get_elements_from_link(link):
    content = requests.get(link).content
    return etree.HTML(content)

In [40]:
def get_html_from_link(link):
    return requests.get(link).content

## link.springer.com

In [26]:
closed_springer = 'http://link.springer.com/chapter/10.1007%2F978-3-319-08747-4_9'

In [27]:
open_springer = 'http://link.springer.com/article/10.1007/s11084-015-9435-4'

In [28]:
def get_springer_open_access(element):
    links = element.xpath('//a')
    words = []
    for link in links:
        if 'viewtype' in link.keys():
            if 'webtrekk-track' in link.get('class'):
                words.append(link.get('viewtype'))
    print(words)
    if 'Denial' in words:
        return False
    else:
        return True

In [29]:
def is_springer_open_access(link):
    elem = get_elements_from_link(link)
    return get_springer_open_access(elem)

In [30]:
is_springer_open_access(closed_springer)

['Preview', 'Denial', 'Denial', 'Preview', 'Add to Papers']


False

In [31]:
is_springer_open_access(open_springer)        

['Article pdf download', 'Full text download', 'Article pdf download', 'Full text download', 'Add to Papers']


True

## sciencedirect.com
### Now with Beautiful Soup!
Note - this one is a lot harder because it uses Javascript to dynamically render a lot of the page content. Tricky.

In [32]:
closed_sd_url = 'http://www.sciencedirect.com/science/article/pii/S0167880915001589'
open_sd_url = 'http://www.sciencedirect.com/science/article/pii/S2212671614001024'

In [102]:
soup = BeautifulSoup(get_html_from_link(closed_sd_url))
soup.title

<title>\n The effects of herbivory and nutrients on plant biomass and carbon storage in Vertisols of an East African savanna\n </title>

In [80]:
all_authors = soup.find_all("a", class_="authorName")

[<a class="authorName svAuthor" data-fn="Lucy W." data-ln="Ngatia" data-orcid="" data-pos="1" data-t="a" data-tb="" href="#" id="authname_N37a42aa0N37ba3fa8">Lucy W. Ngatia</a>,
 <a class="authorName svAuthor" data-fn="Benjamin L." data-ln="Turner" data-orcid="" data-pos="2" data-t="a" data-tb="" href="#" id="authname_N37a42aa0N37ba405c">Benjamin L. Turner</a>,
 <a class="authorName svAuthor" data-fn="Jesee T." data-ln="Njoka" data-orcid="" data-pos="3" data-t="a" data-tb="" href="#" id="authname_N37a42aa0N37ba4110">Jesee T. Njoka</a>,
 <a class="authorName svAuthor" data-fn="Truman P." data-ln="Young" data-orcid="" data-pos="4" data-t="a" data-tb="" href="#" id="authname_N37a42aa0N37ba41c4">Truman P. Young</a>,
 <a class="authorName svAuthor" data-fn="K. Ramesh" data-ln="Reddy" data-orcid="" data-pos="5" data-t="a" data-tb="" href="#" id="authname_N37a42aa0N37ba4278">K. Ramesh Reddy</a>]

In [86]:
x['data-ln']
type(x.attrs)
x.attrs
x.text

u'Lucy W. Ngatia'

In [97]:
one = {'name': 'cat'}
two = {'style': 'classy'}



three = one.update(two)
three
one
two

{'style': 'classy'}

In [110]:
def parse_author_list(author_list):
    all_authors = []
    for author in author_list:
        author_info = author.attrs
        author_info['full_name'] = author.text
        all_authors.append(author_info)
    
    return all_authors

In [111]:
all_authors = soup.find_all("a", class_="authorName")
parse_author_list(all_authors)

[{'class': ['authorName', 'svAuthor'],
  'data-fn': 'Lucy W.',
  'data-ln': 'Ngatia',
  'data-orcid': '',
  'data-pos': '1',
  'data-t': 'a',
  'data-tb': '',
  'full_name': u'Lucy W. Ngatia',
  'href': '#',
  'id': 'authname_N37a42aa0N37ba3fa8'},
 {'class': ['authorName', 'svAuthor'],
  'data-fn': 'Benjamin L.',
  'data-ln': 'Turner',
  'data-orcid': '',
  'data-pos': '2',
  'data-t': 'a',
  'data-tb': '',
  'full_name': u'Benjamin L. Turner',
  'href': '#',
  'id': 'authname_N37a42aa0N37ba405c'},
 {'class': ['authorName', 'svAuthor'],
  'data-fn': 'Jesee T.',
  'data-ln': 'Njoka',
  'data-orcid': '',
  'data-pos': '3',
  'data-t': 'a',
  'data-tb': '',
  'full_name': u'Jesee T. Njoka',
  'href': '#',
  'id': 'authname_N37a42aa0N37ba4110'},
 {'class': ['authorName', 'svAuthor'],
  'data-fn': 'Truman P.',
  'data-ln': 'Young',
  'data-orcid': '',
  'data-pos': '4',
  'data-t': 'a',
  'data-tb': '',
  'full_name': u'Truman P. Young',
  'href': '#',
  'id': 'authname_N37a42aa0N37ba41c4'}

In [106]:
all_authors[0].attrs.update({'full_name': all_authors[0].text})